### Run the following chunk of code to import any necessary libraries or packages required to run the rest of the code.

In [ ]:
import pandas as pd
import numpy as np
import hoomd, hoomd.md as md
from hoomd import azplugins
import gsd, gsd.hoomd, gsd.pygsd
import warnings
import IPython
import packaging.version
import math
import sys

### Run the following chunk of code to continue a previously completed simulation.

The following code can be used to continue a previously completed simulation. Note in this code, the input .gsd file must be present in BOTH the input and output directories. First we initialize the parameters of the simulation to be continued. The first category of parameters are those that we explicitly vary in our experiments. The second set of parameters are those that we did not actively vary, but could in principle be varied in experiments. The final category of parameters are those calculated based upon those defined in the first two categories. After initialization of parameters, the forces which govern the system are defined.

Note, the user will need to adjust their directory structure to match that of the code, or change the code directory variables to match the users directory structure. 

In [ ]:
base_Directory = ""

# These are properties we actively varied in experiments.
phos = "0_x"
working_Input_Directory = base_Directory + "Inputs/" + phos.replace("_","") + "_Phos/"
working_Output_Directory = base_Directory + "Outputs/" + phos.replace("_","") + "_Phos/"
protein_Concentration = 18 #uM
salt_Concentration = 0.000 #M

# These are properties the user may wish to change for experimental purposes, but were kept constant for our paper results.
time_Step = 0.01 #ps
production_Steps = 5*10**8
temperature = 295 #Kelvin
harmonic_Bond_Length = 0.38 #nm
yukawa_Cutoff_Distance = 3.5 #nm
ashbaugh_Cutoff_Distance = 2.0 #nm
LJ_Epsilon_Base = 0.1875 #kCal/mol
LJ_Epsilon_Max = 0.1905 #kCal/mol
monopotassium_Phosphate_Concentration = 0.006085 #mol/L
dipotassium_Phosphate_Concentration = 0.01392 #mol/L
harmonic_Spring_Constant = 8305

# These are properties which are either physical constants, or values which are determined via calculations using previously 
# defined variables.
objects_Per_Mole = 6.02*10**(23)
boltzmann_Constant = 1.38*10**(-23) #J/K
boltzmann_Constant_Converted = boltzmann_Constant*(1/(1*10**(3)))*objects_Per_Mole # kJ/mol K
water_Dielectric_Constant = 78.4
electron_Charge = 1.602*10**(-19)
vacuum_Permittivity = 8.85*10**(-12) #C^2/(Nm^2)
vacuum_Permittivity_Converted = vacuum_Permittivity*((1/(1*10**9))*((1*10**3)/1)*(1/(objects_Per_Mole))) #C^2 mol/kJ nm
yukawa_Epsilon_Scaling = (electron_Charge)**(2)/(4*np.pi*water_Dielectric_Constant*vacuum_Permittivity_Converted) #kJ nm/mol
LJ_Potential_Epsilon = LJ_Epsilon_Base + ((LJ_Epsilon_Max - LJ_Epsilon_Base)/1.250)*salt_Concentration#kCal/mol
LJ_Potential_Epsilon_Converted = LJ_Potential_Epsilon*(4.184/1) #kJ/mol
bjerrum_Length = (electron_Charge**2)/(4*np.pi*vacuum_Permittivity*water_Dielectric_Constant*boltzmann_Constant*temperature) #m
bjerrum_Length_Converted = bjerrum_Length*((1*10**(9))/(1)) #nm
debye_Sum_Scaling = bjerrum_Length_Converted*4*np.pi
buffer_Sum = dipotassium_Phosphate_Concentration*(1**2) + dipotassium_Phosphate_Concentration*(1**2) + \
dipotassium_Phosphate_Concentration*((-1)**2) + dipotassium_Phosphate_Concentration*((-1)**2) + \
monopotassium_Phosphate_Concentration*(1**2) + monopotassium_Phosphate_Concentration*((-1)**2) #mol/L
buffer_Sum_Converted = buffer_Sum*objects_Per_Mole*((1)/(1*10**(24))) #ions/nm^3
salt_Sum = salt_Concentration*(1**2) + salt_Concentration*((-1)**2)
salt_Sum_Converted = salt_Sum*objects_Per_Mole*((1)/(1*10**(24))) #ions/nm^3
debye_Length = 1/(np.sqrt(debye_Sum_Scaling*(buffer_Sum_Converted + salt_Sum_Converted))) #nm
kappa = 1/debye_Length #1/nm

aa_Stats = pd.read_csv(working_Input_Directory + "Amino_Acid_Coarse_Grained_Stats.csv")
aa_Short_Names = aa_Stats["Amino Acid Short"]



c = hoomd.context.initialize("");
sim = hoomd.init.read_gsd(working_Input_Directory + phos.replace("_","") + "_Phos/5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_Trajectory.gsd"
                          ,frame = -1)
snapshot = sim.take_snapshot(particles = True, bonds = True)

cell = hoomd.md.nlist.cell()
cell.reset_exclusions(exclusions=['bond', 'body'])

yukawa = hoomd.md.pair.yukawa(r_cut=0.0, nlist=cell)

for aa_One in aa_Short_Names:
  charge_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Charge"].iloc[0]
  for aa_Two in aa_Short_Names:
    charge_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Charge"].iloc[0]
    yukawa.pair_coeff.set(aa_One,aa_Two,epsilon=charge_One*charge_Two*yukawa_Epsilon_Scaling,kappa = kappa,
                          r_cut = yukawa_Cutoff_Distance)
    
ashbaugh = azplugins.pair.ashbaugh(r_cut=0, nlist=cell)
for aa_One in aa_Short_Names:
  sigma_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Sigma"].iloc[0]/10 #nm
  lambda_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Lambda"].iloc[0]
  for aa_Two in aa_Short_Names:
    sigma_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Sigma"].iloc[0]/10 #nm
    lambda_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Lambda"].iloc[0]
    ashbaugh.pair_coeff.set(aa_One,aa_Two,lam=(lambda_One+lambda_Two)/2.,
                      epsilon=LJ_Potential_Epsilon_Converted, sigma=(sigma_One+sigma_Two)/2.,r_cut=ashbaugh_Cutoff_Distance)
    
harmonic=hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('Amino_Acid_Bond',k=harmonic_Spring_Constant,r0=harmonic_Bond_Length)

hoomd.md.integrate.mode_standard(dt=time_Step) # Time units in ps
kTinput=temperature * boltzmann_Constant_Converted
integrator = hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=kTinput, seed=0)

for aa_One in aa_Short_Names:
  integrator.set_gamma(aa_One,aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Mass"].iloc[0]/1000)

hoomd.dump.gsd(working_Output_Directory + phos.replace("_","") + "_Phos/5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_Trajectory.gsd", period=10**5, group=hoomd.group.all(), truncate=False, overwrite = False)

hoomd.run_upto(production_Steps)